In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:

# Define the path to the dataset
DATASET_PATH= r"D:\working\python project\dataset"

# Emotions mapping (subfolder names)
EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [3]:
def load_data(folder):
    images = []
    labels = []
    for label, emotion in enumerate(EMOTIONS):
        emotion_folder = os.path.join(folder, emotion)
        for file in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, file)
            # Load and preprocess image
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img_resized = cv2.resize(img, (48, 48))  # Resize to 48x48
            images.append(img_resized)
            labels.append(label)
    return np.array(images), np.array(labels)

In [4]:
# Load training and testing data
train_images, train_labels = load_data(os.path.join(r"D:\working\python project\dataset", 'train'))
test_images, test_labels = load_data(os.path.join(r"D:\working\python project\dataset", 'test'))

In [5]:
# Normalize images (0-1 range)
train_images = train_images / 255.0
test_images = test_images / 255.0

In [6]:

# Flatten images for ML model
train_images_flat = train_images.reshape(train_images.shape[0], -1)
test_images_flat = test_images.reshape(test_images.shape[0], -1)

In [7]:
# Train a Random Forest classifier
emotion_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
emotion_classifier.fit(train_images_flat, train_labels)

RandomForestClassifier(random_state=42)

In [8]:
# Test the classifier
predictions = emotion_classifier.predict(test_images_flat)
accuracy = accuracy_score(test_labels, predictions)

print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 46.74%


In [9]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

In [10]:
# Replace with your actual credentials (as strings)
client_id = '4544b915381b425e84dd963824ea5978'
client_secret = '911e485e7d7c47c1b08d50d65ffd5abf'

# Authenticate with Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [11]:
emotion_to_genre = {
    0: 'rock',       # Angry
    1: 'classical',  # Disgust
    2: 'ambient',    # Fear
    3: 'happy',      # Happy
    4: 'chill',      # Neutral
    5: 'acoustic',   # Sad
    6: 'party'       # Surprise
}


In [12]:
def recommend_music(emotion):
    genre = emotion_to_genre.get(emotion, 'chill')  # Default to 'chill' if not mapped
    results = sp.search(q=f'genre:{genre}', type='track', limit=1)
    
    if results['tracks']['items']:
        song = results['tracks']['items'][0]
        song_name = song['name']
        artist_name = song['artists'][0]['name']
        song_uri = song['uri']
        return song_name, artist_name, song_uri
    else:
        return None, None, None

In [13]:
# Define play_music function
import webbrowser
def play_music(song_uri):
    if song_uri:
        webbrowser.open(f"https://open.spotify.com/track/{song_uri.split(':')[2]}")
        print(f"Playing song on Spotify: {song_uri}")
    else:
        print("Song URI is empty or invalid.")

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect face using OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    face_detected = False
    emotion_label = None
    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (48, 48)).reshape(1, -1)
        face_normalized = face_resized / 255.0
        
        # Predict emotion
        emotion = emotion_classifier.predict(face_normalized)
        emotion_label = emotion[0]
        
        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, f'Emotion: {EMOTIONS[emotion_label]}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        face_detected = True  # Indicate that a face was detected

    # Display the video feed
    cv2.imshow('Facial Emotion Recognition', frame)

    # Capture photo if 'c' is pressed and a face is detected
    if cv2.waitKey(1) & 0xFF == ord('c'):
        if face_detected:
            photo_path = "captured_image.jpg"
            cv2.imwrite(photo_path, face)  # Save the last detected face
            print(f"Photo saved as {photo_path}")

            # Recommend music based on emotion
            if emotion_label is not None:
                song_name, artist_name, song_uri = recommend_music(emotion_label)
                if song_name and artist_name:
                    print(f"Recommended Song: {song_name} by {artist_name}")
                    play_music(song_uri)
                else:
                    print("No song found for the detected emotion.")
            else:
                print("Unable to predict emotion. No song will be played.")

            # Break the loop to stop the camera
            break
        else:
            print("No face detected. Unable to capture photo.")

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Photo saved as captured_image.jpg
Recommended Song: The Logical Song by Scooter
Playing song on Spotify: spotify:track:18kaCWM0OOdSQTnxCcoSU5
